## Translate body text (evaluation) <br>

In the following, we translate the extracted body text of the articles from the evaluation dataset. As previously mentioned, the limitations of Google Translate API require extra care in ordere to split the text (in order not to reach the character limit) and the set of documents (in order not to reach the document limit).

In [2]:
import pandas as pd
from googletrans import Translator
import time

In [3]:
eval_df = pd.read_csv('eval/_EVAL_details_in_df.csv')

The limit over characters (max 15k characters according to the official documentation, but 5k in our tests) can be overcome by subdividing the text in a way that preserves basic text units. <br><br>

Our idea was subdividing the text into paragraphs (thus using `\n\n` as delimiter) until their length is shorter than 5000 characters. However, some articles presented some critical peculiarities (absence of paragraphs, absence of formatting, absence of punctuation) and that is a reason to observe that there is no obvious way to cut the text that will not raise some problems. <br><br>

Other attempts which we made are: cutting the text at the last punctuation symbol before the 5000 character limit, brutally cutting the text at 5000 characters and gluing the parts together after translation.

In [10]:
translator = Translator()

def splitTranslator(file,text,lang,delimiter='\n\n'):
    
    # break down long text into smaller chuck by splitting every time it sees the '\n\n' 
    # and keep it in body list 
    body_list =  text.split(delimiter)
    
    # iterate over each text chunks to translate
    translated_list = []
    
    flag_long_paragraph = False
    
    for s in body_list:
        
        # check if any chunk is still over 5000 char
        if len(s) > 5000:
            flag_long_paragraph = True
            print (file,'is still over limit:')
            continue
            
        # if the chunk is under 5000 char, then translate, and keep in the translated list
        elif len(s) == 0:
            continue
        else:
            try:
                translated = translator.translate(s)
                lang = translated.src
                translated = translated.text
                translated_list.append(translated)
            except Exception:
                print (file, 'cannot translate')
            time.sleep(0.5)
    
    # put all the translated text together and connect them using space
    translated_body = ' '.join(translated_list)
    return translated_body,flag_long_paragraph

## main translation task
count = 0


start = 0
end = len(eval_df)
long_paragraph_list = []

for index, row in eval_df.iterrows():
    
    if index < start or index > end:
        continue
    
    print('--------------',index,'-----------------')
    id1 = str(row['pair_id']).split('_')[0]
    id2 = str(row['pair_id']).split('_')[1]
    body1 = str(row['text1'])
    lang1 = row['url1_lang']
    body2 = str(row['text2'])
    lang2 = row['url1_lang']
    
    if lang1 != 'en':
        print('translating text1 of language ', lang1,' of length ', len(body1),'...')
        if len(body1) > 5000:
            split_text1 = splitTranslator(id1,body1,lang1)
            translated1 = split_text1[0]
            if split_text1[1]:
                long_paragraph_list.append(id1)   
        elif pd.isnull(body1):
            continue
        else:
            translated1 = translator.translate(body1)
            lang1 = translated1.src
            translated1 = translated1.text
    else:
        translated1 = body1.replace("\n", " ")
        
    if lang2 != 'en':
        print('translating text2 of language ', lang2,' of length ', len(body2),'...')
        if len(body2) > 5000:
            split_text2 = splitTranslator2(id2,body2,lang2)
            translated2 = split_text2[0]
            if split_text2[1]:
                long_paragraph_list.append(id2)                            
        elif pd.isnull(body2):
            continue
        else:
            translated2 = translator.translate(body2)
            lang2 = translated2.src
            translated2 = translated2.text
    else:
        translated2 = body2.replace("\n", " ")
    
    eval_df.loc[index, "translated_body1"] = translated1
    eval_df.loc[index, "translated_body2"] = translated2
    
    time.sleep(0.5)
    count += 1
    
    if (count+1)%50 == 0:
        print (count+1, 'rows done.')
        
    
print("The number of articles that have been cut due to absence of paragraphs is ",len(long_paragraph_list))

-------------- 0 -----------------
-------------- 1 -----------------
-------------- 2 -----------------
-------------- 3 -----------------
-------------- 4 -----------------
-------------- 5 -----------------
-------------- 6 -----------------
-------------- 7 -----------------
-------------- 8 -----------------
-------------- 9 -----------------
-------------- 10 -----------------
-------------- 11 -----------------
-------------- 12 -----------------
-------------- 13 -----------------
-------------- 14 -----------------
-------------- 15 -----------------
-------------- 16 -----------------
-------------- 17 -----------------
-------------- 18 -----------------
-------------- 19 -----------------
-------------- 20 -----------------
-------------- 21 -----------------
-------------- 22 -----------------
-------------- 23 -----------------
-------------- 24 -----------------
-------------- 25 -----------------
-------------- 26 -----------------
-------------- 27 -----------------
--

-------------- 206 -----------------
translating text1 of language  ru  of length  1066 ...
translating text2 of language  ru  of length  847 ...
-------------- 207 -----------------
translating text1 of language  ru  of length  1040 ...
translating text2 of language  ru  of length  898 ...
-------------- 208 -----------------
translating text1 of language  ru  of length  586 ...
translating text2 of language  ru  of length  725 ...
-------------- 209 -----------------
translating text1 of language  ru  of length  1007 ...
translating text2 of language  ru  of length  643 ...
-------------- 210 -----------------
translating text1 of language  ru  of length  1227 ...
translating text2 of language  ru  of length  859 ...
-------------- 211 -----------------
translating text1 of language  ru  of length  1997 ...
translating text2 of language  ru  of length  1185 ...
-------------- 212 -----------------
translating text1 of language  ru  of length  730 ...
translating text2 of language  ru

translating text1 of language  zh  of length  1111 ...
translating text2 of language  zh  of length  453 ...
-------------- 312 -----------------
translating text1 of language  zh  of length  307 ...
translating text2 of language  zh  of length  321 ...
-------------- 313 -----------------
translating text1 of language  zh  of length  340 ...
translating text2 of language  zh  of length  967 ...
-------------- 314 -----------------
translating text1 of language  zh  of length  1993 ...
translating text2 of language  zh  of length  1956 ...
-------------- 315 -----------------
translating text1 of language  zh  of length  246 ...
translating text2 of language  zh  of length  547 ...
-------------- 316 -----------------
translating text1 of language  zh  of length  1220 ...
translating text2 of language  zh  of length  751 ...
-------------- 317 -----------------
translating text1 of language  zh  of length  150 ...
translating text2 of language  zh  of length  150 ...
-------------- 318

translating text2 of language  tr  of length  4797 ...
-------------- 427 -----------------
translating text1 of language  tr  of length  1192 ...
translating text2 of language  tr  of length  907 ...
-------------- 428 -----------------
translating text1 of language  tr  of length  1568 ...
translating text2 of language  tr  of length  430 ...
-------------- 429 -----------------
translating text1 of language  tr  of length  802 ...
translating text2 of language  tr  of length  887 ...
-------------- 430 -----------------
translating text1 of language  tr  of length  1075 ...
translating text2 of language  tr  of length  2427 ...
-------------- 431 -----------------
translating text1 of language  tr  of length  1144 ...
translating text2 of language  tr  of length  1597 ...
-------------- 432 -----------------
translating text1 of language  tr  of length  2808 ...
translating text2 of language  tr  of length  1096 ...
-------------- 433 -----------------
translating text1 of language 

translating text1 of language  ru  of length  2207 ...
translating text2 of language  ru  of length  1647 ...
-------------- 544 -----------------
translating text1 of language  ru  of length  1358 ...
translating text2 of language  ru  of length  714 ...
-------------- 545 -----------------
translating text1 of language  ru  of length  1526 ...
translating text2 of language  ru  of length  2029 ...
-------------- 546 -----------------
translating text1 of language  ru  of length  1464 ...
translating text2 of language  ru  of length  2086 ...
-------------- 547 -----------------
translating text1 of language  ru  of length  878 ...
translating text2 of language  ru  of length  860 ...
-------------- 548 -----------------
translating text1 of language  ru  of length  951 ...
translating text2 of language  ru  of length  882 ...
550 rows done.
-------------- 549 -----------------
translating text1 of language  ru  of length  2900 ...
translating text2 of language  ru  of length  2909 ..

...

translating text2 of language  ar  of length  581 ...
-------------- 4295 -----------------
translating text1 of language  ar  of length  16293 ...
translating text2 of language  ar  of length  1618 ...
-------------- 4296 -----------------
translating text1 of language  ar  of length  908 ...
translating text2 of language  ar  of length  1439 ...
-------------- 4297 -----------------
translating text1 of language  ar  of length  815 ...
translating text2 of language  ar  of length  2121 ...
-------------- 4298 -----------------
translating text1 of language  ar  of length  1285 ...
translating text2 of language  ar  of length  1287 ...
4300 rows done.
-------------- 4299 -----------------
translating text1 of language  ar  of length  1182 ...
translating text2 of language  ar  of length  3448 ...
-------------- 4300 -----------------
translating text1 of language  ar  of length  1066 ...
translating text2 of language  ar  of length  638 ...
-------------- 4301 -----------------
transl

-------------- 4417 -----------------
translating text1 of language  de  of length  2879 ...
translating text2 of language  de  of length  1127 ...
-------------- 4418 -----------------
translating text1 of language  de  of length  1364 ...
translating text2 of language  de  of length  1713 ...
-------------- 4419 -----------------
translating text1 of language  de  of length  1703 ...
translating text2 of language  de  of length  1078 ...
-------------- 4420 -----------------
translating text1 of language  de  of length  1663 ...
translating text2 of language  de  of length  1569 ...
-------------- 4421 -----------------
translating text1 of language  de  of length  908 ...
translating text2 of language  de  of length  1688 ...
-------------- 4422 -----------------
translating text1 of language  de  of length  983 ...
translating text2 of language  de  of length  756 ...
-------------- 4423 -----------------
translating text1 of language  de  of length  1208 ...
translating text2 of l

translating text2 of language  es  of length  3174 ...
-------------- 4605 -----------------
translating text1 of language  es  of length  2459 ...
translating text2 of language  es  of length  1845 ...
-------------- 4606 -----------------
translating text1 of language  es  of length  2924 ...
translating text2 of language  es  of length  1924 ...
-------------- 4607 -----------------
translating text1 of language  es  of length  754 ...
translating text2 of language  es  of length  2108 ...
-------------- 4608 -----------------
translating text1 of language  es  of length  1037 ...
translating text2 of language  es  of length  838 ...
-------------- 4609 -----------------
translating text1 of language  es  of length  2487 ...
translating text2 of language  es  of length  1752 ...
-------------- 4610 -----------------
translating text1 of language  es  of length  5216 ...
translating text2 of language  es  of length  2993 ...
-------------- 4611 -----------------
translating text1 of 

translating text1 of language  es  of length  4323 ...
translating text2 of language  es  of length  3258 ...
-------------- 4725 -----------------
translating text1 of language  es  of length  933 ...
translating text2 of language  es  of length  1005 ...
-------------- 4726 -----------------
translating text1 of language  es  of length  1513 ...
translating text2 of language  es  of length  2517 ...
-------------- 4727 -----------------
translating text1 of language  es  of length  1634 ...
translating text2 of language  es  of length  1597 ...
-------------- 4728 -----------------
translating text1 of language  es  of length  1743 ...
translating text2 of language  es  of length  1795 ...
-------------- 4729 -----------------
translating text1 of language  es  of length  1235 ...
translating text2 of language  es  of length  947 ...
-------------- 4730 -----------------
translating text1 of language  es  of length  4049 ...
translating text2 of language  es  of length  5067 ...
----

translating text2 of language  zh  of length  573 ...
-------------- 4831 -----------------
translating text1 of language  zh  of length  612 ...
translating text2 of language  zh  of length  263 ...
-------------- 4832 -----------------
translating text1 of language  zh  of length  241 ...
translating text2 of language  zh  of length  188 ...
-------------- 4833 -----------------
translating text1 of language  zh  of length  428 ...
translating text2 of language  zh  of length  513 ...
-------------- 4834 -----------------
translating text1 of language  zh  of length  777 ...
translating text2 of language  zh  of length  1247 ...
-------------- 4835 -----------------
translating text1 of language  zh  of length  316 ...
translating text2 of language  zh  of length  294 ...
-------------- 4836 -----------------
translating text1 of language  zh  of length  1891 ...
translating text2 of language  zh  of length  2911 ...
-------------- 4837 -----------------
translating text1 of language

One can observe that only 37 documents have been truncated due to the absence of formatting or paragraphs. <br><br>

In [9]:
eval_df.loc[2249, "translated_body1"]

'Vienna (DPA-AFX) - The Vienna Stock Exchange has closed on Monday with price losses. The ATX fell 27.81 points or 0.88 percent to 3119.10 units. The European environment was not a clear direction at the beginning of the week. In addition, investors are mainly concerned with the economic impact of the virus crisis and follow themselves accordingly on the market, commented on a Börsian.\n\n\n\nEconomic data from Europe did not provide any significant impulses. The reporting situation on the domestic companies remained quite thin - the reporting season takes place again in the further weekly.\n\n\n\nThe shares in the spotlight counted UNIQA, which, with plus 7.92 percent, also made the winning list in the Prime Market clearly. The insurance company has purchased by a billion euros in Eastern Europe. From the French industry-colleague AXA, the daughters were acquired in Poland, Czech Republic and Slovakia. The acquisition brings five million new customers and 800 million euros multi-premi

In [10]:
path = 'eval/_EVAL_text_translated.csv'
eval_df.to_csv(path,index=False)